In [1]:
import os, sys, numpy as np
import histogram.hdf as hh
import mcvine, mcvine.components

import warnings
warnings.filterwarnings('ignore', module='numpy')
warnings.filterwarnings('ignore')

from mcvine import run_script

In [2]:
sys.path.insert(0, os.path.expanduser('~/git/Qikr/notebooks/production'))
sys.path.insert(0, os.path.expanduser('~/git/Qikr/'))
import plot_utils

In [3]:
workdir = os.path.expanduser("~/simulations/mcvine/qikr")
!mkdir -p {workdir}
%cd {workdir}

/SNS/users/m2d/simulations/mcvine/qikr


# Create instrument

In [4]:
import sample, detector
reload(sample)
reload(detector)

<module 'detector' from '/SNS/users/m2d/git/Qikr/detector.pyc'>

In [215]:
from refl1d.names import *

def calculate_reflectivity(q_values, quartz=False):    
    _zipped = zip(range(len(q_values)), q_values)
    indices, q_values = zip(*sorted(_zipped, key=lambda a:a[1]))
    
    q_values = np.asarray(q_values)
    
    # Fake data array
    zeros = np.zeros(len(q_values))
    # Q-resolution array
    dq = q_values * 0#.02 + 0.0001

    probe = QProbe(q_values, dq, data=(zeros, zeros))

    if quartz:
        quartz = SLD(name='quartz', rho=4.189, irho=0.0)
        air = SLD(name='layer', rho=0.0, irho=0.0)
        sample = ( quartz(0, 0) | air )
    else:
        si = SLD(name='Si', rho=2.01, irho=0.0)
        ir = SLD(name='Ir', rho=7.0, irho=0.0)
        air = SLD(name='layer', rho=0.0, irho=0.0)
        sample = ( si(0, 0) | ir(150.0, 0) | air )

    probe.intensity=Parameter(value=1.0, name='normalization')
    expt = Experiment(probe=probe, sample=sample)
    q, r = expt.reflectivity()
    
    
    _zipped = zip(indices, q_values, r)
    _, q_values, r = zip(*sorted(_zipped, key=lambda a:a[0]))
    
    return r

q = np.logspace(np.log10(0.006), np.log10(0.2), 100)

r = calculate_reflectivity(q)

plot_utils.plot1d([[q,r],], x_log=True, y_log=True)

In [229]:
%%file bsd.py
import sys
import os
sys.path.insert(0, os.path.expanduser('~/git/Qikr/'))
import numpy as np
import os, sys, numpy as np
import histogram.hdf as hh
import mcvine, mcvine.components

from sample import Sample
from detector import Detector

from refl1d.names import *

def calculate_reflectivity(q):
    # Fake data array
    _q_values = np.asarray(q)
    
    _zipped = zip(range(len(_q_values)), _q_values)
    indices, q_values = zip(*sorted(_zipped, key=lambda a:a[1]))
    q_values = np.asarray(q_values)
    
    zeros = np.zeros(len(q_values))
    # Q-resolution array
    dq = np.zeros(len(q_values))

    probe = QProbe(q_values, dq, data=(zeros, zeros))

    if True:
        quartz = SLD(name='quartz', rho=4.189, irho=0.0)
        air = SLD(name='layer', rho=0.0, irho=0.0)
        sample = ( quartz(0, 0) | air )
    else:
        si = SLD(name='Si', rho=2.01, irho=0.0)
        ir = SLD(name='Ir', rho=7.0, irho=0.0)
        air = SLD(name='layer', rho=0.0, irho=0.0)
        sample = ( si(0, 0) | ir(150.0, 0) | air )

    probe.intensity=Parameter(value=1.0, name='normalization')
    expt = Experiment(probe=probe, sample=sample)
    _q, r = expt.reflectivity()

    _zipped = zip(indices, q_values, r)
    _, _q, r = zip(*sorted(_zipped, key=lambda a:a[0]))

    fd = open("qvalues.txt", 'w')
    fd.write("Number of neutrons: %s\n" % str(len(q)))
    fd.write("Q range: %s %s\n" % (np.min(q), np.max(q)))
    fd.write("R range: %s %s\n" % (np.min(r), np.max(r)))
    
    if len(q_values) < 1000:
        for i in range(len(q)):
            if r[i]>0:
                fd.write("R(%s) = %s \n" % (q[i], r[i]))
            
        #for i in range(len(q_values)):
        #    fd.write("%s,\n" % q_values[i])
    fd.close()

    #return 0.5*np.zeros(len(r)) # np.asarray(r)

    return np.asarray(r)

def instrument(theta=4.0, sam2det=1.83, direct_beam=False):
    source_sam_dist = 2.0
    if direct_beam:
        theta = 0.0
    instrument = mcvine.instrument()

    source = mcvine.components.sources.NeutronFromStorage('beam', path='./work_5e8_dec6/beam.neutrons')
    instrument.append(source, position=(0,0,0))

    # Slit 0 #################################################################
    slit_0_width = 0.02
    #slit_0_height = 0.00108
    slit_0_height = 0.02
    #slit_0_height = 0.00400
    slit_0 = mcvine.components.optics.Slit(name='slit_0', width=slit_0_width, height=slit_0_height)
    instrument.append(slit_0, position=(0,0,0.01), relativeTo=source)

    slit_1_width = 0.02
    #slit_1_height = 0.00045
    slit_1_height = 0.02
    #slit_1_height = 0.00300
    slit_1 = mcvine.components.optics.Slit(name='slit_1', width=slit_1_width, height=slit_1_height)
    instrument.append(slit_1, position=(0,0,source_sam_dist-0.1), relativeTo=source)
    
    instrument.append(mcvine.components.monitors.NeutronToStorage('save_pre', 'pre_detector.neutrons'),
                      position=(0,0,source_sam_dist-0.01), relativeTo=source)

    sample_position = mcvine.components.optics.Arm()
    instrument.append(sample_position, position=(0,0,source_sam_dist), relativeTo=source)

    if not direct_beam:
        sample = Sample('sample', xwidth=20000.02, zheight=20000.02, rq=calculate_reflectivity)
        instrument.append(sample, position=(0,0,0), orientation=(-theta, 0, 0), relativeTo=sample_position)

    instrument.append(mcvine.components.monitors.NeutronToStorage('save', 'detector.neutrons'),
                      position=(0,0,0.0001), relativeTo=sample_position)

    from detector import Detector
    det = Detector('detector', xwidth=0.1792, yheight=0.2128, dx=0.0007, dy=0.0007, outfile='events.npy')
    twotheta_radian = np.deg2rad(2*theta)
    instrument.append(
        det,
        position=(0, sam2det*np.sin(twotheta_radian), sam2det*np.cos(twotheta_radian)),
        orientation=(-2*theta, 0, 0),
        relativeTo=sample_position)
    

    return instrument

Overwriting bsd.py


In [701]:
# neutrons = ns.readneutrons_asnpyarr('./work_5e8_dec6/beam.neutrons')
neutrons = ns.readneutrons_asnpyarr('./work_5e8_dec6/beam.neutrons')
print neutrons.shape
#total = np.sum(neutrons.T[-1])
print "Total weight = %s" % total 
print "Total neutrons = %s" % (total/neutrons.shape[0])


theta = neutrons.T[4]/neutrons.T[5]


theta_dist, bins = np.histogram(theta, bins=400, weights=neutrons.T[-1])

plot_utils.plot1d([[bins, theta_dist],], y_log=False)

(218566955, 10)
Total weight = 2.23170098544e+17
Total neutrons = 1021060564.91


# Run a test simulation
work_5e8_dec6/beam.neutrons contains 218,566,955 neutrons.

Total sum of weights = 2.23170098544e+17

Total number of neutrons = 1,021,060,564.91


In [88]:
import sample, detector
reload(sample)
reload(detector)

<module 'detector' from '/SNS/users/m2d/git/Qikr/detector.pyc'>

In [230]:
%%time
n_pulses = 1
!rm -rf ./debug-bsd/
#run_script.run_mpi('./bsd.py', workdir='./debug-bsd', ncount=int(1e7), overwrite_datafiles=False, nodes=10)
run_script.run_mpi('./bsd.py', workdir='./debug-bsd', ncount=int(n_pulses*218566955), overwrite_datafiles=False, nodes=10)
#run_script.run1('./bsd.py', workdir='./debug-bsd', ncount=int(55000), overwrite_datafiles=False)

pulse_flux = 218566955

CPU times: user 47.1 ms, sys: 1.34 s, total: 1.39 s
Wall time: 2min 46s


In [194]:
import glob
files = glob.glob('./debug-bsd/rank*-step*/events.npy')
events = np.load(files[0])
for f in files[1:]: events = np.concatenate((events, np.load(f)))

np.save('./events.npy', events)

neutrons is a numpy array of shape (N,10). N is number of neutrons.

The 10 floats for a neutron are (x,y,z,vx,vy,vz,s1,s2,t,prob)

In [162]:
from mcni import neutron_storage  as ns
from mcni.utils import conversion

def _process(f, binning=None, theta=None, db=False): 
    neutrons = ns.readneutrons_asnpyarr(f)
    
    tof = neutrons.T[8]
    vx = neutrons.T[3]
    vy = neutrons.T[4]
    vz = neutrons.T[5]
    h = 6.626e-34  # m^2 kg s^-1
    m = 1.675e-27  # kg
    speed = np.sqrt(vx**2+vy**2+vz**2)
    dist = tof * speed
    
    #dist = 18.1
    cst = dist / h * m
    wl  = tof / cst * 1e10

    #wl = 2.0*np.pi/conversion.V2K/speed
    
    # Cut wl range as we would do in analysis
    neutrons.T[-1][wl<2.5] = 0.0
    neutrons.T[-1][wl>22] = 0.0

    if theta is None:
        #q = 2.0*conversion.V2K*speed * vy/speed/ 2.0
        
        #wl = 2.0*np.pi/conversion.V2K/speed
        # We are in lab space and vy/vz gives us two-theta
        speed = np.sqrt(vz**2+vy**2)
        theta = np.arcsin(vy/speed) * 180.0 / np.pi / 2.0

        #theta = np.arcsin(vy/vz) * 180.0 / np.pi / 2.0
        q = 4.0*np.pi/wl * np.sin(np.pi/180.0*theta)
        
        print("theta = %s" % (np.average(theta, weights=neutrons.T[-1])))
    else:
        theta_i = np.arcsin(vy/vz) * 180.0 / np.pi / 2.0
        theta_f = theta_i - theta
        if db:
            theta_f = 0
            theta += theta_i
            q = 4.0*np.pi/wl * np.sin(np.pi/180.0*theta) 
        else:
            q = 4.0*np.pi/wl * np.sin(np.pi/180.0*(theta+theta_f) * np.cos(np.pi/180.0*theta_f))

    #histo, bins = np.histogram(1000.0*neutrons.T[4]/neutrons.T[5], bins=400, weights=neutrons.T[-1])
    if binning is None:
        binning=150

    speed = np.sqrt(vz**2+vy**2)
    theta = np.arcsin(vy/speed) * 180.0 / np.pi / 2.0
    
    histo, bins = np.histogram(q, bins=binning, range=[0,0.4], weights=neutrons.T[-1])
    #histo, bins = np.histogram(q, bins=binning, range=[0,0.25], weights=neutrons.T[-1])
    histo, bins = np.histogram(wl, bins=binning, range=[0,25], weights=neutrons.T[-1])
    #histo, bins = np.histogram(theta, bins=binning, range=[3.9,4.1], weights=neutrons.T[-1])
    #histo, bins = np.histogram(theta, bins=binning, range=[7.8,8.1], weights=neutrons.T[-1])
    #histo, bins = np.histogram(theta, bins=binning, range=[-.021, .021], weights=neutrons.T[-1])
    histo2d, bins_x, bins_y = np.histogram2d(dist, wl, range=[[17.9, 19.0], [1, 23]], bins=binning, weights=neutrons.T[-1])
    #histo, bins = np.histogram(dist, bins=binning, weights=neutrons.T[-1])
    return bins, histo, histo2d, bins_x, bins_y


bins, sc, sc2d, bx, by = _process('./debug-bsd/detector.neutrons', theta=None)
bins, db, sc2d, bx, by = _process('./debug-bsd/pre_detector.neutrons', theta=1, db=True)

plot_utils.plot_heatmap(bx, by, np.log(sc2d).T)

#plot_utils.plot1d([[bins, sc]], y_log=True)
plot_utils.plot1d([[bins, sc],[bins, db]], y_log=True)

q_mid = [(bins[i+1]+bins[i])/2.0 for i in range(len(bins)-1)]
_r = calculate_reflectivity(q_mid)

plot_utils.plot1d([[bins, sc/db],[bins, _r]], y_log=True)


theta = 3.99999832779


# Make a 2D histogram of the events

In [13]:
index = events['pixelID']
Nx, Ny = 256, 304
yindex = index % Ny
xindex = index // Ny


ixy, xbbs, ybbs = np.histogram2d(xindex, yindex, bins=(np.arange(Nx), np.arange(Ny)), weights=events['p'])

In [14]:
print ixy.shape
plot_utils.plot_heatmap(np.arange(256), np.arange(304), np.log(ixy).T)

(255, 303)


# Convert to nexus file

In [15]:
from mantid2mcvine.nxs import template as nxs_template, Events2Nxs

idf_path = os.path.expanduser("~/git/Qikr/QIKR_Definition.xml")
nxs_template.create(idfpath=idf_path, 
                    ntotpixels = Nx*Ny, outpath='template.nxs', pulse_time_end=1e5)

e2n = Events2Nxs.Event2Nxs('./template.nxs', nmonitors=0)

In [195]:
#e2n.run('./events.npy', 'scattered_beam_108_45_1pulse_2x2cm_sample.nxs')
#e2n.run('./events.npy', 'scattered_beam_400_300_1pulse_2x2cm_sample.nxs')
#e2n.run('./events.npy', 'direct_beam_108_45_1pulse.nxs')
#e2n.run('./events.npy', 'scattered_beam_200_90.nxs')
#e2n.run('./events.npy', 'scattered_beam_108_45_4deg.nxs')


#e2n.run('./events.npy', 'scattered_beam_quartz_108_45_1pulse_2x2cm_sample.nxs')
#e2n.run('./events.npy', 'scattered_beam_quartz_108_45_4deg.nxs')
#e2n.run('./events.npy', 'direct_beam_200_90.nxs')

Converting ./events.npy to scattered_beam_quartz_108_45_4deg.nxs


# Analysis


In [227]:
import mantid
import mantid.simpleapi as api

def load_data(run='scattered_beam.nxs'):
    ws = api.Load(run, OutputWorkspace=run)
    ws = api.Rebin(InputWorkspace=ws, Params='6000,10,50000', PreserveEvents=True, OutputWorkspace=run)
    ws = api.ConvertUnits(InputWorkspace=ws, Target="Wavelength", OutputWorkspace=run)
    print(str(ws))
    return ws

def mantid(sc, db, theta=1):
    sc = api.Load(sc, OutputWorkspace=sc)
    db = api.Load(db, OutputWorkspace=db)

    # Scaling TOF to take into account delay in effective emission time
    # 18./18.1 = 0.994475138121547
    sc = api.ScaleX(sc, Factor=1/1.006)
    db = api.ScaleX(db, Factor=1/1.006)

    
    api.AddSampleLog(sc, LogName='thi', LogText='0', LogType='Number Series', LogUnit='deg')
    api.AddSampleLog(sc, LogName='tthd', LogText='%s' % 2.0*theta, LogType='Number Series', LogUnit='deg')

    api.AddSampleLog(db, LogName='thi', LogText='0', LogType='Number Series', LogUnit='deg')
    api.AddSampleLog(db, LogName='tthd', LogText='%s' % 2.0*theta, LogType='Number Series', LogUnit='deg')

    refl_sc = api.LiquidsReflectometryReduction(InputWorkspace=sc, NormFlag=0, OutputWorkspace="refl_sc",
                                            TOFRangeFlag=True, TOFRange=[12500,115000],
                                            QMin=0.005, QStep=-0.02, SignalPeakPixelRange=[50,200])
    refl_db = api.LiquidsReflectometryReduction(InputWorkspace=db, NormFlag=0, OutputWorkspace="refl_db",
                                            TOFRangeFlag=True, TOFRange=[12500,115000],
                                            QMin=0.005, QStep=-0.02, SignalPeakPixelRange=[50,200])

    refl_output = refl_sc/refl_db

    #api.SaveAscii(InputWorkspace=refl_output,Filename='/SNS/users/m2d/simulations/mcvine/qikr/qikr_reduced_1pulse_%s_deg.txt' % theta)
    #LRReflectivityOutput(ReducedWorkspaces=['refl_output',], ScaleToUnity=False, OutputFilename='/SNS/users/m2d/simulations/mcvine/qikr/qikr_reduced_1pulse.txt')
    return refl_output

In [18]:
def get_peak(center, width):
    peak_min = int(round(float(center) - float(width)/2.0))
    peak_max = int(round(float(center) + float(width)/2.0+1.0))
    return peak_min, peak_max

In [19]:
theta = 1. * np.pi/180.

q1 = 4*np.pi/2.5 * np.sin(theta)
q0 = 4*np.pi/23.0 * np.sin(theta)

print q0, q1


wl_min = 2.5
wl_max = 23.0



h = 6.626e-34  # m^2 kg s^-1
m = 1.675e-27  # kg
dist = 19.85

cst = dist / h * m
#wl  = tof / cst * 1e10

t_min = wl_min * cst / 10000
t_max = wl_max * cst / 10000


print t_min, t_max


0.00953536553928 0.0877253629613
12544.8045578 115412.201932


In [214]:
#ws_sc = load_data('scattered_beam_108_44.nxs')
#ws_db = load_data('direct_beam_108_44.nxs')

#ws_sc = load_data('scattered_beam_400_300.nxs')
#ws_sc = load_data('scattered_beam_400_300_1pulse_2x2cm_sample.nxs')

ws_sc = load_data('scattered_beam_108_45_1pulse_2x2cm_sample.nxs')

#ws_sc = load_data('scattered_beam_108_45_4deg.nxs')
#ws_sc = load_data('scattered_beam_108_45_8deg.nxs')
#ws_db = load_data('direct_beam_108_45_1pulse.nxs')

# QUARTZ
#ws_sc = load_data('scattered_beam_quartz_108_45_1pulse_2x2cm_sample.nxs')
#ws_sc = load_data('scattered_beam_quartz_108_45_4deg.nxs')

#ws_sc = load_data('scattered_beam_200_90.nxs')
#ws_db = load_data('direct_beam_200_90.nxs')

scattered_beam_108_45_1pulse_2x2cm_sample.nxs


In [228]:
_refl_ws = mantid('scattered_beam_108_45_1pulse_2x2cm_sample.nxs',
                  'direct_beam_108_45_1pulse.nxs', theta=1)
q = _refl_ws.readX(0)
r = _refl_ws.readY(0)
dr = _refl_ws.readDx(0)

_r = calculate_reflectivity(q, quartz=False)

data = np.loadtxt('qikr_Ir_150A_1pulse_1_deg.txt')


plot_utils.plot1d([[q,r,dr], [q,_r]], data_names=['Mantid', 'Theory'], y_log=True)

In [309]:
wl = np.arange(2, 25, 0.1)
wl_dist = compute_wl2d(ws_sc, wl)

_wl_dist = fuzz(wl_dist)

plot_utils.plot_heatmap(wl, np.arange(304), np.log(_wl_dist), x_title='Wavelength [1/A]', y_title="Pixel")

/SNS/software/miniconda2/envs/py2/lib/python2.7/site-packages/ipykernel/__main__.py:6: RuntimeWarning:

divide by zero encountered in log

/SNS/software/miniconda2/envs/py2/lib/python2.7/site-packages/ipykernel/__main__.py:6: RuntimeWarning:

invalid value encountered in log



In [20]:
def fuzz(dist, err=None):
    dist = np.asarray(dist)
    rnd = np.random.randn(*dist.shape)
    if err is None:
        err = np.sqrt(dist)
    return dist + err*rnd


In [21]:
def compute_wl2d(ws, wl_bins):
    n_x = 256
    n_y = 304
    
    pixel_width = .0007
    det_distance = 1.83


    wl_dist = np.zeros([n_y, len(wl_bins)-1])
    
    for i in range(n_y):
        for j in range(n_x):
            pixel = j*n_y + i
            evt_list = ws.getSpectrum(pixel)
            if evt_list.getNumberEvents() == 0:
                continue

            wl_list = evt_list.getTofs()
            w_list = evt_list.getWeights()
            
            # Cut wl range as we would do in analysis
            w_list[wl_list<2.5] = 0.0
            w_list[wl_list>22] = 0.0
            
            
            _wl_dist, _ = np.histogram(wl_list, bins=wl_bins, weights=w_list)
            wl_dist[i] += _wl_dist
            
    wl_dist /= pulse_flux
    return wl_dist

In [142]:
def compute2(ws, q_bins, theta, x_pos=150, x_w=40, y_pos=120.5, y_w=87, pulse_flux=218566955):
    n_x = 256
    n_y = 304
    
    pixel_width = .0007
    det_distance = 1.83

    refl = np.zeros(len(q_bins)-1)
    refl_count = np.zeros(len(q_bins)-1)
    print refl.shape


    y_min, y_max = get_peak(y_pos, y_w)
    x_min, x_max = get_peak(x_pos, x_w)

    x_dist = np.zeros(n_x)
    y_dist = np.zeros(n_x)
    
    for i in range(y_min, y_max):
        for j in range(x_min, x_max):
            pixel = j*n_y + i
            evt_list = ws.getSpectrum(pixel)
            if evt_list.getNumberEvents() == 0:
                continue

            wl_list = evt_list.getTofs() * 18.0/18.15
            w_list = evt_list.getWeights()
            
            # Cut wl range as we would do in analysis
            w_list[wl_list<2.5] = 0.0
            w_list[wl_list>22] = 0.0


            x_distance = (i - y_pos) * pixel_width
            theta_f = np.arcsin(x_distance/det_distance)/2.0
            
            #print("%s %s %s %s" % (i, j, x_distance, theta_f))
            theta_f = 0
            qz=4.0*np.pi/wl_list * np.sin(theta + theta_f) * np.cos(theta_f)
            
            _refl, bins = np.histogram(qz, bins=q_bins, weights=w_list)
            refl += _refl
            
            x_dist[j] += evt_list.getNumberEvents()
            y_dist[i] += evt_list.getNumberEvents()


    refl /= pulse_flux
    x_dist /= pulse_flux
    y_dist /= pulse_flux
    return refl, x_dist, y_dist

In [216]:
q = np.logspace(np.log10(0.008), np.log10(0.12), 150)
q = np.logspace(np.log10(0.008), np.log10(0.4), 150)


print q[1]/q[0]
#theta = 1.0 * np.pi/180.
#theta = 4.03 * np.pi/180.
theta = 1.0 * np.pi/180.


r, x_sc, y_sc = compute2(ws_sc, q, theta, x_pos=126, x_w=75, y_pos=150, y_w=140)
n, x_db, y_db = compute2(ws_db, q, theta, x_pos=126, x_w=75, y_pos=150, y_w=100)

r_fuzz = fuzz(r)
n_fuzz = fuzz(n)

#refl = r_fuzz/n_fuzz
refl = r/n

d_refl = np.sqrt( r/n**2 + r / n**3)

print refl.shape, q.shape

q_mid = [(q[i+1]+q[i])/2.0 for i in range(len(q)-1)]
_r = calculate_reflectivity(q_mid)

#plot_utils.plot1d([[q_mid, r, np.sqrt(r)], [q_mid, n, np.sqrt(n)]], y_log=True, x_log=True)
plot_utils.plot1d([[q_mid, refl, d_refl], [q_mid, _r]],
                  data_names=['1 pulse', 'theory'],
                  y_log=True, x_log=True)

plot_utils.plot1d([[np.arange(304), np.log(x_sc)],
        [np.arange(256), np.log(y_sc)],
        [np.arange(304), np.log(x_db)],
        [np.arange(256), np.log(y_db)],
       ], data_names=['sc x', 'sc y', 'db x', 'db y'])

1.02660289174
(149,)
(149,)
(149,) (150,)


In [203]:
import time

# 1 pulse
r_fuzz = fuzz(r)
n_fuzz = fuzz(n)

refl_1p = r_fuzz/n_fuzz
d_refl_1p = np.sqrt( r_fuzz/n_fuzz**2 + r_fuzz / n_fuzz**3)

# 1 second
_scale = 7.5
r_fuzz = fuzz(_scale*r, np.sqrt(_scale*r))
n_fuzz = fuzz(_scale*n, np.sqrt(_scale*n))

refl_1s = r_fuzz/n_fuzz
d_refl_1s = np.sqrt( r_fuzz/n_fuzz**2 + r_fuzz / n_fuzz**3)

# 1 min
_scale = 7.5*60.0
r_fuzz = fuzz(_scale*r, np.sqrt(_scale*r))
n_fuzz = fuzz(_scale*n, np.sqrt(_scale*n))

refl_1m = r_fuzz/n_fuzz
d_refl_1m = np.sqrt( r_fuzz/n_fuzz**2 + r_fuzz / n_fuzz**3)

# 10 min
_scale = 7.5*60.0*10.0
r_fuzz = fuzz(_scale*r, np.sqrt(_scale*r))
n_fuzz = fuzz(_scale*n, np.sqrt(_scale*n))

refl_10m = r_fuzz/n_fuzz
d_refl_10m = np.sqrt( r_fuzz/n_fuzz**2 + r_fuzz / n_fuzz**3)


q_mid = [(q[i+1]+q[i])/2.0 for i in range(len(q)-1)]
_r = calculate_reflectivity(q_mid)

plot_utils.plot1d([[q_mid, refl_1p, d_refl_1p],
                   [q_mid, refl_1s, d_refl_1s], 
                   [q_mid, refl_1m, d_refl_1m], 
                   [q_mid, refl_10m, d_refl_10m], 
                   [q_mid, _r]],
                  data_names=['1 pulse', '1 sec', '1 min', '10 min', 'theory'],
                  y_log=True, x_log=True)


# Save data
sample = "quartz" #"Ir"
thickness = "" #"150A"
theta = "4"

header = "# QIKR %s\n" % time.ctime()
header += "# %s %s  2cm x 2cm\n" % (sample, thickness)
header += "# Footprint = 30mm\n"

data = np.asarray(zip(q_mid, refl_1p, d_refl_1p))
np.savetxt('qikr_%s_%s_1pulse_%s_deg.txt' % (sample, thickness, theta), data, header=header)

data = np.asarray(zip(q_mid, refl_1s, d_refl_1s))
np.savetxt('qikr_%s_%s_1sec_%s_deg.txt' % (sample, thickness, theta), data, header=header)

data = np.asarray(zip(q_mid, refl_1m, d_refl_1m))
np.savetxt('qikr_%s_%s_1min_%s_deg.txt' % (sample, thickness, theta), data, header=header)

data = np.asarray(zip(q_mid, refl_10m, d_refl_10m))
np.savetxt('qikr_%s_%s_10min_%s_deg.txt' % (sample, thickness, theta), data, header=header)


In [208]:
plot_data = []
data_names = []

time = "1min"
sample = "quartz" #"Ir"
thickness = "" #"150A"

data = np.loadtxt('qikr_%s_%s_%s_1_deg.txt' % (sample, thickness, time))
plot_data.append([data.T[0], data.T[1], data.T[2]])
data_names.append('Theta = 1 degree')
q_min = data.T[0][0]


data = np.loadtxt('qikr_%s_%s_%s_4_deg.txt' % (sample, thickness, time))
plot_data.append([data.T[0], data.T[1], data.T[2]])
data_names.append('Theta = 4 degrees')
q_max = data.T[0][-1]

q = np.logspace(np.log10(q_min), np.log10(q_max), 150)
_r = calculate_reflectivity(q)
plot_data.append([q, _r])
data_names.append('theory')

plot_utils.plot1d(plot_data, data_names=data_names, x_log=True, y_log=True)
